# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

We will create 3 different Agents - Planner , Writer , Editor

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False, ### This agent is not allowed to delegte their task to other agents
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

So we have created agents but we dont have any task to guide them . So we will create 3 Tasks - Plan,Write,Edit

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
### What do you expext your Planner Agent to do 

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

By default crew operates sequentially . So order of tasks list is important . 

Here plan will execute first followed by write and edit (Because output of plan will be sent as an input to write . And output of write will be sent as an input to edit)

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the rapid growth and impact of artificial intelligence in various industries
- Introduction to the latest trends, key players, and noteworthy news in the AI field

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing developments
- AI in healthcare, finance, and transportation industries
- Ethical considerations in AI development

2. Key 

I now can give a great answer.

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial intelligence (AI) has been rapidly growing and making a significant impact across various industries. From healthcare to finance and transportation, AI is revolutionizing the way businesses operate and how we interact with technology. In this blog post, we will explore the latest trends, key players, and noteworthy news in the AI field, shedding light on the advancements that are shaping the future of AI technology.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous advancements in machine learning. This technology allows machines to learn from data and improve their performance over time, making AI systems more intelligent and efficient. Additionally, natural language processing (NLP) developments have enabled AI to understand and generate human language, leading to breakthroughs in areas suc

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial intelligence (AI) has been rapidly growing and making a significant impact across various industries. From healthcare to finance and transportation, AI is revolutionizing the way businesses operate and how we interact with technology. In this blog post, we will explore the latest trends, key players, and noteworthy news in the AI field, shedding light on the advancements that are shaping the future of AI technology.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous advancements in machine learning. This technology allows machines to learn from data and improve their performance over time, making AI systems more intelligent and efficient. Additionally, natural language processing (NLP) developments have enabled AI to understand and generate human language, leading to breakthroughs in areas such as virtual assistants and language translation.

AI is also making waves in industries like healthcare, finance, and transportation. In healthcare, AI-powered diagnostics are improving accuracy and efficiency in disease detection, leading to better patient outcomes. In finance, AI is being used for fraud detection and risk management, enhancing security and compliance measures. In transportation, AI is driving innovations in autonomous vehicles and smart traffic management systems, revolutionizing the way we commute.

As AI technology continues to advance, ethical considerations have become a crucial aspect of AI development. Questions around data privacy, bias in AI algorithms, and the societal impact of AI systems are hot topics in the field. It is essential for AI developers and users to navigate these ethical dilemmas responsibly to ensure that AI technology is used for the greater good of society.

## Key Players in Artificial Intelligence

Several key players are driving innovation and shaping the AI industry. Google Brain, known for its research in deep learning and AI applications, has been at the forefront of AI advancements. OpenAI, a research organization focused on developing safe AI, is pioneering breakthroughs in machine learning and AI ethics. IBM Watson, Microsoft Azure AI, and NVIDIA are also major players in the AI space, providing cutting-edge AI solutions to businesses and consumers worldwide.

## Noteworthy News in Artificial Intelligence

Recent news in the AI field has highlighted the impact of AI-powered healthcare diagnostics, with breakthroughs in disease detection and treatment. AI-driven customer service improvements are also making waves, with virtual assistants and chatbots enhancing customer experiences. Ethical debates surrounding AI decision-making processes are sparking discussions on the responsible use of AI technology, while AI's role in sustainability and climate change solutions is offering new possibilities for addressing environmental challenges.

## Ethical Considerations in AI Development

As AI technology becomes more pervasive in our daily lives, addressing ethical considerations in AI development is critical. From ensuring transparent and accountable AI decision-making processes to mitigating bias in AI algorithms, ethical guidelines are essential for the responsible deployment of AI systems. Organizations and policymakers must collaborate to establish ethical frameworks that prioritize the well-being of individuals and society at large.

## Call to Action

In conclusion, staying informed on the latest AI trends and engaging in discussions on AI ethics is crucial for professionals in technology, business leaders, and students interested in AI. By understanding the latest trends, key players, and ethical considerations in AI development, we can harness the power of AI technology for positive societal impact. Let's continue to explore the potential of AI and work towards a future where AI innovation is driven by ethical principles and human values.

---
This blog post offers a comprehensive overview of the current landscape of artificial intelligence, covering the latest trends, key players, noteworthy news, and ethical considerations in the AI industry. By incorporating insights from credible sources and addressing the interests and pain points of the target audience, this article aims to provide valuable information and spark meaningful conversations around the future of AI technology.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).